1.Install Lib

In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import httpx

client = OpenAI(
    base_url="", 
    api_key="",
    http_client=httpx.Client(
        base_url="",
        follow_redirects=True,
    ),
)
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Output only vaild JSON"},
    {"role": "user", "content": "Input"}
  ]
)

print(completion)

ChatCompletion(id='chatcmpl-wIEkg0oXq1lCn4OBAF2iQyUL1hp1N', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To provide valid JSON output, I need an input to work with. Could you please provide the input you'd like to convert into valid JSON?", role='assistant', function_call=None, tool_calls=None))], created=1713975209, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=9, total_tokens=38))


In [3]:
import requests
url = 'https://newsapi.org/v2/top-headlines'
parameters = {
    'country': 'us',
    'apiKey': ''
}

response = requests.get(url, params=parameters)

print(response.json())

{'status': 'ok', 'totalResults': 36, 'articles': [{'source': {'id': None, 'name': 'Thenationalnews.com'}, 'author': 'Nicky Harley', 'title': "Patients 'less likely to die' if treated by a female doctor, study reveals - The National", 'description': 'Researchers find lower mortality and hospital readmission rates in those treated by female physicians', 'url': 'https://www.thenationalnews.com/health/2024/04/23/patients-less-likely-to-die-if-treated-by-a-female-doctor-study-reveals/', 'urlToImage': 'https://thenational-the-national-prod.cdn.arcpublishing.com/resizer/v2/IV7OQQPEVM33VTX4GBQ7EZ4NOQ.jpg?smart=true&auth=e30a7c24f4f543db5221258768f4081a1176fc6b36c06fcc0962faf496726899&width=1200&height=630', 'publishedAt': '2024-04-23T14:17:48Z', 'content': 'Patients treated by female physicians have a reduced risk of dying, with female patients benefiting more than their male counterparts, new research has found.\r\nThe mortality rate for female patients … [+5070 chars]'}, {'source': {'id': 'r

In [4]:
!pip install ipython

In [5]:
import time
from datetime import date,timedelta
import json
import pandas as pd
pd.set_option('display.max_colwidth',None)
from IPython.display import display

import os
import requests

from scipy import spatial


2. build prompt of queries using GPT-3.5

In [6]:
def get_completion_gpt(input, gpt_model="gpt-3.5-turbo-1106"):

    # Make the API request
    completion = client.chat.completions.create(
    model= gpt_model,
    messages=[
    {"role": "system", "content": "Output only vaild JSON"},
    {"role": "user", "content": input},
    ],
    response_format={"type":"json_object"}
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed

In [7]:
user_query = "What are the events that impacted the bitcoin price lately?."

input = f""" 
You have access to a search NEWS API that returns recent news articles.
Make a list of search questions that match this topic.
Try using different keywords that are related to the topic to make your
questions more general.

Create many queries, some with specific terms and others without them.
Be imaginative and make as many queries as you can. More queries will help you
find better results.

Pick 10 of these queries.
For example, you can include queries like ['keyword_1 keyword_2', 'keyword_1',
'keyword_2'].

# User question: {user_query}

# Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

In [8]:
gpt_model = "gpt-3.5-turbo"

In [9]:
parsed = get_completion_gpt(input, gpt_model)
parsed

{'queries': ['bitcoin price recent events',
  'bitcoin market news',
  'cryptocurrency price updates',
  'crypto market trends',
  'blockchain technology developments',
  'digital currency market analysis',
  'bitcoin impact on financial market',
  'latest news on cryptocurrency',
  'virtual currency market updates',
  'bitcoin price movement']}

In [10]:
queries = parsed['queries']
queries.append(user_query)
queries

['bitcoin price recent events',
 'bitcoin market news',
 'cryptocurrency price updates',
 'crypto market trends',
 'blockchain technology developments',
 'digital currency market analysis',
 'bitcoin impact on financial market',
 'latest news on cryptocurrency',
 'virtual currency market updates',
 'bitcoin price movement',
 'What are the events that impacted the bitcoin price lately?.']

4. Fetch news articles from NEWS API for each query:

In [34]:
import requests

def NEWS_API_KEY():
    apiKey = ''
    return apiKey

def search_news(query, num_articles=5, from_datetime="2024-04-01", to_datetime="2024-04-25"):
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": NEWS_API_KEY(),
            "pageSize": num_articles,
            "sortBy": "relevancy",
            "from": from_datetime,
            "to": to_datetime,
        }
    )
    return response.json()

In [35]:
articles = []
for query in queries:
    result = search_news (query)
    if result['status'] == 'ok':
        articles = articles + result ['articles']
    else:
        raise Exception (result ["message"])

len (articles)

50

In [36]:
# remove duplicates
articles = list({article["url"]:article for article in articles}.values())
len(articles)

43

5. Re-rank

In [40]:
hypoth_answer =f"""
"Make up
an answer to the user's question. We'll use this made-up answer tou sort the search results.
Imagine you have all the details to answer, but don't use real facts.
Instead, use things like 'EVENT affected something' or 'NAME mentionedu
"something on DATE' as placeholders.
User question: {user_query}
Format: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""

In [43]:
parsed_hypothet_answer = get_completion_gpt (hypoth_answer)
parsed_hypothet_answer

{'hypotheticalAnswer': 'Recently, a major EVENT affected the bitcoin price, leading to a significant increase in value. This was followed by another event where NAME mentioned something on DATE, causing a slight dip in the price. Overall, the events that impacted the bitcoin price lately have been quite dynamic.'}

In [46]:
hypoth_answer_11m = parsed_hypothet_answer ['hypotheticalAnswer']
hypoth_answer_11m

'Recently, a major EVENT affected the bitcoin price, leading to a significant increase in value. This was followed by another event where NAME mentioned something on DATE, causing a slight dip in the price. Overall, the events that impacted the bitcoin price lately have been quite dynamic.'

In [ ]:
6.Embedding

In [58]:
def get_embeddings (input) :
    response = client. embeddings.create(model="text-embedding-ada-002",input=input)
    return [data.embedding for data in response.data]

In [59]:
# 6.1 The hypotheticalAnswer

In [60]:
embedding_hypoth = get_embeddings (hypoth_answer_11m)[0]
embedding_hypoth [: 5]

[-0.026865005493164062,
 -0.028626644983887672,
 0.02339353784918785,
 -0.015556832775473595,
 -0.024365030229091644]

In [ ]:
# 6.2 The content articles

In [65]:
articles_prepare_embedd = [
    f"{article['title']} {article['description']} {article['content'][0:100]}"
    for article in articles
    ]
print(f"Length of articles to embed: {len(articles_prepare_embedd)}")

Length of articles to embed: 43


In [69]:
article_embeddings = get_embeddings (articles_prepare_embedd)
print(f"Length of embedding vector: {len(article_embeddings)}")
article_embeddings[0][:5]

Length of embedding vector: 43


[0.012298943474888802,
 -0.017819151282310486,
 0.007637730333954096,
 0.00030360312666743994,
 -0.011393336579203606]

7. Against Hypothetical Answer == Re-Rank

7.1 Calculate consin similarity

In [73]:
from scipy import spatial

similarity_score_func=lambda x, y: 1 - spatial.distance.cosine(x, y)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(similarity_score_func(embedding_hypoth,article_embedding))
    
cosine_similarities[0:10]

[0.7770616649858723,
 0.8575892412876297,
 0.6997396846677595,
 0.8333097374706964,
 0.8405245735968782,
 0.7377960879828624,
 0.8097797041742064,
 0.7601761156279383,
 0.7587701545882278,
 0.7659980230823105]

In [ ]:
7.2 Get the most relevant articles

In [75]:
scored_articles = zip(articles, cosine_similarities)
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)
print("Top 5 articles:", "\n")

Top 5 articles: 



In [ ]:
8. Against Original User Query

In [79]:
embedding_original_query = get_embeddings(user_query)[0]
embedding_original_query[:5]                                        

[-0.02085202932357788,
 -0.026151368394494057,
 0.021529389545321465,
 -0.014994867146015167,
 -0.03054755926132202]

In [81]:
similarity_score_func=lambda x, y: 1 - spatial.distance. cosine(x, y)
# Calculate cosine similarity
cosine_similarities_original = []
for article_embedding in article_embeddings:
    cosine_similarities_original.append (similarity_score_func(embedding_original_query, article_embedding))

In [82]:
cosine_similarities_original [0:10]

[0.7919108734980538,
 0.8494198264515883,
 0.7239132420819473,
 0.8235745915855325,
 0.8448798794313979,
 0.7710898428195807,
 0.8185226737736092,
 0.7645708728243569,
 0.7780031171657636,
 0.7654987096727417]

The similarity scores are much lower than the one against the hypotethical answer:

In [85]:
scored_articles_original = zip(articles, cosine_similarities_original)
scored_articles_original = sorted(scored_articles_original, key=lambda x: x[1], reverse=True)
print("Top 5 articles:", "\n")
for article, score in scored_articles_original[0:5]:
    print("Title:", article["title"])
    print("Url:", article["url"])
    print("Date of publication:", article["publishedAt"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print("Score:", score)
    print()

Top 5 articles: 

Title: What's Next For Bitcoin Prices After Their Latest Pullback?
Url: https://www.forbes.com/sites/digital-assets/2024/04/17/whats-next-for-bitcoin-prices-after-their-latest-pullback/
Date of publication: 2024-04-18T00:36:47Z
Description: What's next for bitcoin prices after the cryptocurrency repeatedly approached $60,000 earlier today? Several analysts weigh in.
Content: Analysts weighed in on what's next for bitcoin prices after their latest drop. (Photo illustration ....
Score: 0.8500779943352983

Title: Bitcoin passes $65,000 on the big 'halving' day
Url: https://qz.com/bitcoin-bitcoin-halving-cryptocurrency-1851421828
Date of publication: 2024-04-19T14:38:00Z
Description: After days of tumultuous trading, Bitcoin finally reached $65,000 Friday morning, thanks to its big halving event. The jump in Bitcoin price is a relief for investors as giant banks such as JPMorgan and Goldman Sachs were less confident about the top cryptocu…
Content: After days of tumultuou

9.Final Answer

9.1 Final answer with hypothetical re-rank:

In [86]:
from IPython import display

In [87]:
formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

final_input = f"""
Generate an answer to the user's question based on the given search results.
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {user_query}

Include as much information as possible in the answer. Reference the relevant
search result urls as markdown links.
"""

gpt_model = "gpt-3.5-turbo-1106"

completion = client.chat.completions.create(
    model=gpt_model,
    messages=[
        {"role": "user", "content": final_input},
    ],
)

display.display(display.Markdown(completion.choices[0].message.content))

Lately, several events have influenced the price of Bitcoin. One significant factor was the much-anticipated halving event, which occurred recently. This event typically brings about volatility in the cryptocurrency market. Bitcoin reached a high of $65,000 on the day of the halving, providing relief to investors after a period of tumultuous trading. However, prior to this, the price had fallen below $60,000, causing some nervousness among investors. Additionally, the launch of a Bitcoin ETF in Hong Kong towards the end of April also impacted the price, with fluctuations observed around the $70,000 threshold. If you're interested in more details, you can check out these articles: [Bitcoin passes $65,000 on the big 'halving' day](https://qz.com/bitcoin-bitcoin-halving-cryptocurrency-1851421828), [Bitcoin falls below $60,000 as halving nerves set in](https://qz.com/bitcoin-below-60-000-bitcoin-halving-crypto-1851416763), and [Bitcoin Stays Sub $70,000 As Hong Kong Readies April-End ETF Launch](https://biztoc.com/x/bcf5b8f938d38641).

10 Final Thoughts
In this notebook, you have discovered :
• How to use LLMs to generate various queries in regards to the user query ==> Prompt builder with JSON results
• Then, how to use these queries to fetch news from NEW API. (btw, you need an API KEY)
• Also, we will use again LLMs to help us build a Hypothetical Answer against which, we w'll rank the news article. We'll use embedding and cosine similarity to get the most relevant answers. Comparison will be made with the original query.
• Then, we will ask the LLM the original user query and see what happens:
"What are the events that impacted the bicoin price lately?."